In [1]:
using JuMP
using CPLEX

Now we define the master problem :)

In [92]:
function solve_master(A,b,c,h,T,V,R)
    relaxation = 15
    nx = size(c,1)
    m = Model(CPLEX.Optimizer)
    @variable(m, x[1:nx] >= 0)
    @variable(m, theta)
    if(size(V,1) > 0)
        @objective(m, Min, sum(c[i]*x[i] for i = 1:nx) + theta)
    else
        @objective(m, Min, sum(c[i]*x[i] for i = 1:nx))
    end
    @constraint(m, A*x .== b)
    if size(R,1) > 0
        for i in 1:size(R,1)
            @constraint(m, transpose(R[i,:])*(h-T*x)<=0)
        end
    end
    if size(V,1) > 0
        for i in 1:size(V,1)
            @constraint(m, theta >= transpose(V[i,:])*(h-T*x))
        end
    end
    optimize!(m)
    sol = zeros(1, nx)
    for i = 1:nx
        sol[i] = JuMP.value(x[i])
    end
    sol = vec(sol)
    if(size(V,1)>0)
        return (objective_value(m), sol, theta,termination_status(model))
    else
        return (objective_value(m), sol, -Inf,termination_status(model))
    end
end

solve_master (generic function with 1 method)

In [93]:
A = [1 1; -1 0;0 -1]
b = [120; -40; -20]
c =[100; 150]

solve_master(A,b,c,[],[],[],[])

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Infeasibility row 'c1':  0  = 60.
Presolve time = 0.00 sec. (0.00 ticks)


LoadError: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

In [74]:
A

3×2 Matrix{Int64}:
  1   1
 -1   0
  0  -1

In [75]:
for i in 1:size(A,1)
    print(A[i,:])
end

[1, 1][-1, 0][0, -1]